# Output Parsers

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:

- `get_format_instructions() -> str`: A method which returns a string containing instructions for how the output of a language model should be formatted.
- `parse(str) -> Any`: A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

Below we go over some examples of output parsers.

In [1]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

## PydanticOutputParser
This output parser allows users to specify an arbitrary JSON schema and query LLMs for JSON outputs that conform to that schema.

Keep in mind that large language models are leaky abstractions! You'll have to use an LLM with sufficient capacity to generate well-formed JSON. In the OpenAI family, DaVinci can do reliably but Curie's ability already drops off dramatically. 

Use Pydantic to declare your data model. Pydantic's BaseModel like a Python dataclass, but with actual type checking + coercion.

In [2]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

In [3]:
model_name = 'text-davinci-003'
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [4]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    
    def example_query() -> str:
        return "Tell me a joke."

data_model = Joke

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=data_model)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

_input = prompt.format_prompt(query=data_model.example_query())

output = model(_input.to_string())

parser.parse(output)

Joke(setup='Why did the chicken cross the playground?', punchline='To get to the other slide!')

In [5]:
# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")
        
    def example_query() -> str:
        return "Generate the filmography for a random actor."

data_model = Actor

parser = PydanticOutputParser(pydantic_object=data_model)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

_input = prompt.format_prompt(query=data_model.example_query())

output = model(_input.to_string())

parser.parse(output)

Actor(name='Tom Hanks', film_names=['Forrest Gump', 'Saving Private Ryan', 'The Green Mile', 'Cast Away', 'Toy Story', 'A League of Their Own'])

<br>
<br>
<br>
<br>

---

## Structured Output Parser

While the Pydantic/JSON parser is more powerful, we initially experimented data structures having text fields only.

In [6]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

Here we define the response schema we want to receive.

In [7]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

We now get a string that contains instructions for how the response should be formatted, and we then insert that into our prompt.

In [8]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

We can now use this to format a prompt to send to the language model, and then parse the returned result.

In [9]:
model = OpenAI(temperature=0)

In [10]:
_input = prompt.format_prompt(question="what's the capital of france")
output = model(_input.to_string())

In [11]:
output_parser.parse(output)

{'answer': 'Paris', 'source': 'https://en.wikipedia.org/wiki/Paris'}

And here's an example of using this in a chat model

In [12]:
chat_model = ChatOpenAI(temperature=0)

In [13]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("answer the users question as best as possible.\n{format_instructions}\n{question}")  
    ],
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [14]:
_input = prompt.format_prompt(question="what's the capital of france")
output = chat_model(_input.to_messages())

In [15]:
output_parser.parse(output.content)

{'answer': 'Paris', 'source': 'https://en.wikipedia.org/wiki/Paris'}

## CommaSeparatedListOutputParser

Here's another parser strictly less powerful than Pydantic/JSON parsing.

In [16]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [17]:
output_parser = CommaSeparatedListOutputParser()

In [18]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

In [19]:
model = OpenAI(temperature=0)

In [20]:
_input = prompt.format(subject="ice cream flavors")
output = model(_input)

In [21]:
output_parser.parse(output)

['Vanilla',
 'Chocolate',
 'Strawberry',
 'Mint Chocolate Chip',
 'Cookies and Cream']